# Laboratorio 4: NLP
### Data Science - Sección 20
Pablo Andrés Zamora Vásquez - 21780 <br>
Diego Andrés Morales Aquino - 21762

## Importación de datos y preprocesamiento

In [4]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parámetros del preprocesamiento
max_features = 50000  # Usar las 50,000 palabras más frecuentes
maxlen = 200  # Longitud máxima de secuencias (rellenaremos a esta longitud)

# Cargar los datos del dataset IMDB
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Secuenciar y rellenar las reseñas para tener una longitud uniforme
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Mostrar la forma de los datos procesados
print(f"Train data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

Train data shape: (25000, 200)
Test data shape: (25000, 200)


In [7]:
# Función para calcular la longitud de cada reseña
def get_review_lengths(reviews):
    return np.array([len(review) for review in reviews])

# Función para calcular la proporción de palabras positivas/negativas
# Simplificar al contar las palabras en función de su índice
def get_pos_neg_ratio(reviews, positive_threshold=1000):
    positive_ratios = []
    for review in reviews:
        positive_words = sum(1 for word in review if word < positive_threshold)
        ratio = positive_words / len(review) if len(review) > 0 else 0
        positive_ratios.append(ratio)
    return np.array(positive_ratios)

# Características adicionales
train_review_lengths = get_review_lengths(X_train)
test_review_lengths = get_review_lengths(X_test)

train_pos_neg_ratios = get_pos_neg_ratio(X_train)
test_pos_neg_ratios = get_pos_neg_ratio(X_test)

# Concatenar estas características adicionales a las secuencias originales
X_train_additional = np.column_stack((X_train, train_review_lengths, train_pos_neg_ratios))
X_test_additional = np.column_stack((X_test, test_review_lengths, test_pos_neg_ratios))

print(f"Forma de datos de entrenamiento con características adicionales: {X_train_additional.shape}")
print(f"Forma de datos de prueba con características adicionales: {X_test_additional.shape}")

Forma de datos de entrenamiento con características adicionales: (25000, 202)
Forma de datos de prueba con características adicionales: (25000, 202)


## Modelo

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate

# Parámetros del modelo
embedding_size = 128
lstm_units = 128
dropout_rate = 0.5

# Entrada para las secuencias de palabras (reseñas)
input_seq = Input(shape=(maxlen,), name="input_sequence")
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_size, input_length=maxlen)(input_seq)

# Capas LSTM
lstm_layer = LSTM(lstm_units, return_sequences=True)(embedding_layer)
lstm_layer = LSTM(lstm_units)(lstm_layer)
lstm_layer = Dropout(dropout_rate)(lstm_layer)

# Entrada para las características adicionales
input_features = Input(shape=(2,), name="input_features")  # Dos características adicionales: longitud y proporción de palabras positivas/negativas

# Concatenar la salida LSTM con las características adicionales
concatenated = Concatenate()([lstm_layer, input_features])

# Añadir capas densamente conectadas
dense_layer = Dense(64, activation="relu")(concatenated)
dense_layer = Dropout(dropout_rate)(dense_layer)
output_layer = Dense(1, activation="sigmoid")(dense_layer)  # Clasificación binaria (positivo/negativo)

# Definir el modelo
model = Model(inputs=[input_seq, input_features], outputs=output_layer)

# Compilar el modelo
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Resumen del modelo
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_sequence      │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 200, 128)  │  6,400,000 │ input_sequence[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 200, 128)  │    131,584 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 128)       │    131,584 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128)       │          0 │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_features      │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 130)       │          0 │ dropout_4[0][0],  │
│ (Concatenate)       │                   │            │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      8,384 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         65 │ dropout_5[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,671,617 (25.45 MB)

 Trainable params: 6,671,617 (25.45 MB)

 Non-trainable params: 0 (0.00 B)

## Entrenamiento

In [9]:
# Concatenar las características adicionales en un solo array
train_additional_features = np.column_stack((train_review_lengths, train_pos_neg_ratios))
test_additional_features = np.column_stack((test_review_lengths, test_pos_neg_ratios))

# Entrenamiento del modelo
history = model.fit(
    [X_train, train_additional_features],  # Solo dos entradas: secuencias + características combinadas
    y_train,  # Etiquetas
    epochs=5,  # Ajustar según sea necesario
    batch_size=64,
    validation_data=([X_test, test_additional_features], y_test)  # Solo dos entradas en los datos de validación
)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 201s 505ms/step - accuracy: 0.5067 - loss: 3.0819 - val_accuracy: 0.5000 - val_loss: 0.6943
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 219s 561ms/step - accuracy: 0.4978 - loss: 0.6946 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 219s 560ms/step - accuracy: 0.4961 - loss: 0.6944 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 220s 562ms/step - accuracy: 0.4987 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 222s 568ms/step - accuracy: 0.4967 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932


## Evaluación

In [11]:
# Concatenar las características adicionales en un solo array para la evaluación
test_additional_features = np.column_stack((test_review_lengths, test_pos_neg_ratios))

# Evaluación del modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(
    [X_test, test_additional_features],  # Solo dos entradas: secuencias + características combinadas
    y_test,
    verbose=2
)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



# Comparación con un modelo simple

# Modelo simple LSTM sin características adicionales
simple_input_seq = Input(shape=(maxlen,))
simple_embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_size, input_length=maxlen)(simple_input_seq)
simple_lstm_layer = LSTM(lstm_units)(simple_embedding_layer)
simple_lstm_layer = Dropout(dropout_rate)(simple_lstm_layer)
simple_output_layer = Dense(1, activation="sigmoid")(simple_lstm_layer)

simple_model = Model(inputs=simple_input_seq, outputs=simple_output_layer)
simple_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Entrenamiento del modelo simple
simple_history = simple_model.fit(
    X_train, y_train,  # Solo secuencias de palabras
    epochs=5,
    batch_size=64,
    validation_data=(X_test, y_test)
)

# Evaluación del modelo simple
simple_test_loss, simple_test_accuracy = simple_model.evaluate(X_test, y_test, verbose=2)

print(f"Simple Model Test Loss: {simple_test_loss}")
print(f"Simple Model Test Accuracy: {simple_test_accuracy}")


# Comparación de la precisión y la pérdida
print(f"Modelo con características adicionales - Precisión: {test_accuracy}, Pérdida: {test_loss}")
print(f"Modelo simple - Precisión: {simple_test_accuracy}, Pérdida: {simple_test_loss}")


782/782 - 46s - 58ms/step - accuracy: 0.5000 - loss: 0.6932
Test Loss: 0.6931692957878113
Test Accuracy: 0.5
Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 106s 265ms/step - accuracy: 0.7164 - loss: 0.5307 - val_accuracy: 0.8610 - val_loss: 0.3289
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 103s 263ms/step - accuracy: 0.9234 - loss: 0.2117 - val_accuracy: 0.8464 - val_loss: 0.3576
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 102s 262ms/step - accuracy: 0.9473 - loss: 0.1549 - val_accuracy: 0.8670 - val_loss: 0.4032
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 100s 257ms/step - accuracy: 0.9748 - loss: 0.0789 - val_accuracy: 0.8566 - val_loss: 0.4942
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 103s 263ms/step - accuracy: 0.9839 - loss: 0.0515 - val_accuracy: 0.7875 - val_loss: 0.5654
782/782 - 28s - 35ms/step - accuracy: 0.7875 - loss: 0.5654
Simple Model Test Loss: 0.5653678774833679
Simple Model Test Accuracy: 0.7875199913978577
Modelo con características adicionales - Precisión: 0.5, Pérdida: 0.6931692957878113
M